In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import cpi
import re
from transformers import short_list
cpi.update()

In [2]:
data = pd.read_csv(r"../data/IMDb movies.csv",low_memory=False)

In [3]:
def checkIndex(data,index):
    """
    Compare unique index with the amount of rows
    """
    if data[index].nunique() == data.shape[0]:
        print("Each index has 1 row")
    else:
        print(f"Attention! Exists {data[index].nunique()} indexs but the dataset has {data.shape[0]} rows!")
checkIndex(data,index="imdb_title_id")

Each index has 1 row


In [4]:
def time(f):
    import datetime as dt
    def wrapper(data,*args,**kargs):
        tic = dt.datetime.now()
        result = f(data,*args,**kargs)
        toc = dt.datetime.now()
        print(f"{f.__name__} took {toc-tic} shape: {result.shape}")
        return result
    return wrapper

def start_pipeline(data):
    '''
    Fuck the efficiency, this way is easier to maintain
    '''
    return data.copy()

##### Preproccesing steps

In [5]:
@time
def clean_year(data,column):
    '''
    Clean the years and convert it to int32
    '''
    data[column] = data[column].apply(lambda x: re.sub("[^0-9]","",x)
                                      if (isinstance(x,str)) else x)
    data[column] = data[column].astype(int)
    return data
@time
def clean_money_columns(data,columns,currency="$"):
    '''
    Select the currency and convert the values to float
    '''
    data = data.dropna(subset=columns)
    for column in columns:
        #get only the rows with the selected currency
        data = data[data[column].str.startswith(currency)]
        #clean and transform to float
        data[column] = data[column].apply(lambda x: x.replace("$","").strip() 
                                              if(isinstance(x,str)) else x)
        data[column] = data[column].astype(float)
    return data
@time   
def inflation_adjust(data):    
    '''
    Adjust the selected columns by inflation
    '''    
    def adjust_money(x,year):
        try:
            return cpi.inflate(x,year)
        except:
            pass       
     
    data["budget_adj"] = data.apply(
                        lambda x: adjust_money(x["budget"],x["year"]),axis=1)
    data["income_adj"] = data.apply(
                        lambda x: adjust_money(x["worlwide_gross_income"],
                                               x["year"]),axis=1)
    return data
@time
def cat_long_form(data,columns):
    '''
    Transform the given columns into longForm
    param: 
    - columns: list of columns
    '''
    for col in columns:
        data[col] = data[col].apply(
                    lambda x: x.split(",") if isinstance(x,str) else None)
        data[col].dropna(inplace=True)
        data = data.explode(col)
        data[col] = data[col].apply(
                    lambda x: x.strip() if x != None else None)
    return data

@time
def create_columns(data,columns,drop_old=True):
    '''
    Drop nan and create columns depending on the list of values in each column
    '''
    def listToColumns(data,columnToConvert,nombre,cantidad):
        '''
        Transform a cell containing a list of values, to 1 column per value 
        '''
        if cantidad == 1:
            data[columnToConvert] = data[columnToConvert].apply(lambda x: x.split(",")[0])
        else:
            cols = [f'{nombre}{i}' for i in range(cantidad)]
            for i in cols:
                data[i] = ""
            for i in data.index:
                listOfNames = data[columnToConvert].loc[i].split(",")[0:cantidad]
                for j in range(len(listOfNames)):
                    data.loc[i,cols[j]] = listOfNames[j]
            for col in cols:
                data[col] = data[col].apply(lambda x: None if not x else x)
                data[col] = data[col].apply(
                                lambda x: x.strip() if x != None else None)
        return data

    print("Creating new columns...")
    data.dropna(axis="index",subset=COLUMNS_TO_CREATE.keys(),inplace=True)
    
    for column_to_convert,amount in columns.items():
        listToColumns(data,column_to_convert,column_to_convert,amount) 
    data = data.fillna(method="ffill",axis=1)
    return data
@time
def drop_na(data,columns):
    data.dropna(subset=columns,inplace=True)
    return data
@time
def check_type(data,columns):
    '''
    Check if all the rows of the columns provided are strings
    '''
    print("***CHECKING ALL SELECTED COLUMNS ARE STRINGS***")
    for col in columns:
        np.array_equal(data[col],data[col].astype(str))
        print(f'{col}:{np.array_equal(data[col],data[col].astype(str))}')
    return data
@time
def filter_categorical(X,columns):
    '''
    Filter categorical values depending on how many times it appear in the column.
    The values that cant pass the filter will be set to OTHER
    '''
    for column in columns:
        if column in ["language","country","director"]:
            sl = short_list(data=X,column=column,min_amount=5,len_limit=10)
            X.loc[~X[column].isin(sl.index),column] = 'OTHER'
        else:
            sl = short_list(data=X,column=column,min_amount=5)
            X.loc[~X[column].isin(sl.index),column] = 'OTHER'
    return X
@time
def filter_small(X,column="income_adj",quantile=0.1):
    '''
    Filter 0.1 quantile incomes
    '''
    min = X[column].quantile(quantile)
    return X.drop(X[X[column]<min].index)
    
    

#### PIPELINE

In [12]:
#specify how many columns i want for each many to many categorical column
COLUMNS_TO_CREATE = {"actors":5,"director":1,"writer":1,"genre":2,
                     "country":1,"language":1,"production_company":1}
CATEGORICAL_COLUMNS = ['genre', 'country', 'language', 
                       'director', 'writer','production_company', 'actors','date_published']
COLS_TO_ENCODE = ['genre0','writer','language',"country","director",
                  "actors0","production_company","actors1","actors2","actors3","actors4","genre1"]
#pipe
new_data=(data
 .pipe(start_pipeline)
 .pipe(clean_year,"year")
 .pipe(clean_money_columns,["budget","worlwide_gross_income"])
 .pipe(inflation_adjust)
 .pipe(create_columns,COLUMNS_TO_CREATE)
 .pipe(check_type,CATEGORICAL_COLUMNS)
 .pipe(filter_categorical,COLS_TO_ENCODE)
 .pipe(filter_small)
)

#save
new_data.to_csv("preprocessed_data")

clean_year took 0:00:00.078221 shape: (85855, 22)
clean_money_columns took 0:00:00.044739 shape: (9025, 22)
inflation_adjust took 0:00:00.379226 shape: (9025, 24)
Creating new columns...
create_columns took 0:00:04.921645 shape: (8911, 31)
***CHECKING ALL SELECTED COLUMNS ARE STRINGS***
genre:True
country:True
language:True
director:True
writer:True
production_company:True
actors:True
date_published:True
check_type took 0:00:00.016321 shape: (8911, 31)
filter_categorical took 0:00:04.187367 shape: (8911, 31)
filter_small took 0:00:00.008630 shape: (8020, 31)


In [13]:
new_data

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,reviews_from_critics,budget_adj,income_adj,actors0,actors1,actors2,actors3,actors4,genre0,genre1
165,tt0010323,Il gabinetto del dottor Caligari,Das Cabinet des Dr. Caligari,1920,1920-02-27,"Fantasy, Horror, Mystery",76,Germany,OTHER,OTHER,...,160.0,243873.0,119375.8335,OTHER,OTHER,OTHER,OTHER,OTHER,Fantasy,Horror
245,tt0012190,I quattro cavalieri dell'Apocalisse,The Four Horsemen of the Apocalypse,1921,1923-04-16,"Drama, Romance, War",150,USA,OTHER,OTHER,...,16.0,12110391.061453,139022339.263129,OTHER,OTHER,OTHER,OTHER,OTHER,Drama,Romance
251,tt0012349,Il monello,The Kid,1921,1923-11-26,"Comedy, Drama, Family",68,USA,English,OTHER,...,105.0,3784497.206704,407454.107263,OTHER,OTHER,OTHER,OTHER,OTHER,Comedy,Drama
348,tt0014624,La donna di Parigi,A Woman of Paris: A Drama of Fate,1923,1927-06-06,"Drama, Romance",82,USA,OTHER,OTHER,...,24.0,5562015.789474,178000.351462,OTHER,OTHER,OTHER,OTHER,OTHER,Drama,Romance
415,tt0015864,La febbre dell'oro,The Gold Rush,1925,1925-10-23,"Adventure, Comedy, Drama",95,USA,English,OTHER,...,125.0,14291732.0,416767.344,OTHER,OTHER,OTHER,OTHER,OTHER,Adventure,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85397,tt9354944,Jexi,Jexi,2019,2019-10-11,"Comedy, Romance",84,USA,English,OTHER,...,42.0,5299483.292067,9901368.041086,OTHER,OTHER,Rose Byrne,OTHER,OTHER,Comedy,Romance
85460,tt9426210,Weathering with You,Tenki no ko,2019,2019-10-14,"Animation, Drama, Family",112,Japan,Japanese,OTHER,...,110.0,11764852.908389,204747634.5245,OTHER,OTHER,OTHER,OTHER,OTHER,Animation,Drama
85530,tt9527778,Doblemente Embarazada,Doblemente Embarazada,2019,2019-12-20,Comedy,98,OTHER,Spanish,OTHER,...,4.0,1006901.825493,1023391.697704,OTHER,OTHER,OTHER,OTHER,OTHER,Comedy,Comedy
85740,tt9779516,Cosa mi lasci di te,I Still Believe,2020,2020-03-19,"Biography, Drama, Music",116,USA,English,OTHER,...,52.0,12563762.745787,14324285.128066,OTHER,OTHER,OTHER,OTHER,OTHER,Biography,Drama


In [14]:

##### STILL WORKING ON THIS
from difflib import SequenceMatcher,get_close_matches

def get_similar(x,serie):
    '''
    Check for similar strings inside a column
    return: all the strings that are similar
    '''
    serie = list(serie.dropna().unique())
    similarities = get_close_matches(x,serie,cutoff=0.8)
    if len(similarities) > 1 : 
        return {f"{x}":similarities}